# 分类猫和狗的图片

- 3.6B参数 = 14GB 
<img src="./pic/分类猫和狗的图片.PNG" width=300 height=300>

## 两个原则
- 平移不变性
- 局部性

## 重新考察全连接层
- 空间信息，用矩阵表示
- <img src="./pic/重新考察全连接层1.PNG" width=400 height=400>
- 原来的k,l是基于图片的绝对位置，ab是根据ij的相对位置

## 原则 #1 - 平移不变性
- 当在图片中形成一个识别器后，在一定像素大小的范围内，它都有自己的权重，当这个识别器在图片上换位置之后，他的权重应该不变
- 卷积核不会随着位置变化而变化
- <img src="./pic/平移不变性.PNG" width=400 height=400>
- 卷积就是weight shared全连接
- 当模型的取值范围受限，模型的复杂度降低

## 原则 #2 - 局部性

- h<sub>i,j</sub>的结果只需要在x<sub>i,j</sub>输入附近的那些点
- 当这些点与i,j的距离超过Δ时，权重为0（不再关注）。Δ为移动单位（步长stride）
- <img src="./pic/局部性.PNG" width=400 height=400>

## 总结
- 所以卷积层是一个特殊的全连接层
- 卷积层不会像全连接层一样，随着层数的增大，而导致参数变得特别巨大
- <img src="./pic/全连接层到卷积层.PNG" width=400 height=400>

## 以上只是讲解卷积操作子

# 2. 卷积层
<img src="./pic/卷积层.PNG" width=400 height=400>


## 二维交叉相关
<img src="./pic/二维交叉相关.PNG" width=400 height=400>
<img src="./pic/二维交叉相关(卷积核).gif" width=400 height=400>

## 二维卷积层
- w = kernel，是一个可学习的参数
<img src="./pic/二维卷积层.PNG" width=400 height=400>
- 神经网络通过学习这样的kernel，来得到想要的输出
<img src="./pic/不同卷积核的效果图.PNG" width=400 height=400>

## 交叉相关与卷积
<img src="./pic/交叉相关与卷积.PNG" width=400 height=400>

## 一维和三维交叉相关
<img src="./pic/一维和三维交叉相关.PNG" width=400 height=400>


## 总结
 
- 卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出
- 核矩阵和偏移是可以学习的参数
- 核矩阵的大小是超参数


# 2. 图像卷积

- 互相关运算

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):    # X为输入，K为核矩阵
    """计算二维互相关运算"""
    h, w = K.shape    # 返回行数和列数
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    # 输入的高度：X.shape[0] - h + 1
    # 输入的宽度：X.shape[1] - w + 1
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()    # 哈达玛积(不是数学上传统的矩阵乘积)，再求和
    return Y

### 验证上述二维互相关运算的输出

In [17]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

## 输入一个3个通道的张量，对应一个3维卷积核
## 输出n个通道的张量，对应n个3维卷积核

In [26]:
# 输入3维张量，输出1维张量

X = torch.arange(27.).reshape(3, 3, 3)
K = torch.arange(12.).reshape(3, 2, 2)
output_channel = 1

X, K, corr2d(X[0], K[0]) , corr2d(X[1], K[1]), corr2d(X[2], K[2])

(tensor([[[ 0.,  1.,  2.],
          [ 3.,  4.,  5.],
          [ 6.,  7.,  8.]],
 
         [[ 9., 10., 11.],
          [12., 13., 14.],
          [15., 16., 17.]],
 
         [[18., 19., 20.],
          [21., 22., 23.],
          [24., 25., 26.]]]),
 tensor([[[ 0.,  1.],
          [ 2.,  3.]],
 
         [[ 4.,  5.],
          [ 6.,  7.]],
 
         [[ 8.,  9.],
          [10., 11.]]]),
 tensor([[19., 25.],
         [37., 43.]]),
 tensor([[249., 271.],
         [315., 337.]]),
 tensor([[767., 805.],
         [881., 919.]]))

In [27]:
corr2d(X[0], K[0])+corr2d(X[1], K[1])+corr2d(X[2], K[2])

tensor([[1035., 1101.],
        [1233., 1299.]])

### 实现二维卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 卷积层的一个简单应用：通过找到像素变化的位置，来检测图像中不同颜色的边缘

In [5]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [6]:
K = torch.tensor([[1.0, -1.0]])

- 输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘，其他情况的输出为 0

In [7]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

- 学习由X生成Y的卷积核

In [10]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {loss.sum():.3f}')

epoch 2, loss 9.939
epoch 4, loss 1.673
epoch 6, loss 0.283
epoch 8, loss 0.048
epoch 10, loss 0.008


- 所学的卷积核的权重张量
    - 几乎接近 K = torch.tensor([[1.0, -1.0]])

In [11]:
conv2d.weight.data.reshape((1, 2))

tensor([[ 0.9862, -0.9811]])

In [12]:
K

tensor([[ 1., -1.]])